In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    env = FrozenLakeCustom(
        map_name=args["map_name"], is_slippery=args["is_slippery"],
        render_mode=args["render_mode"])

    simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [ ]:
args["update_method"] = "max"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")

Q_mcts_dict = defaultdict()



hparam_ucb_scale_list = np.arange(10, 100, 10)
# hparam_ucb_scale_list = np.arange(20, 64, 4)
hparam_ucb_scale_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]


num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
# num_trajectories_list = [200, 500, 800]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
for num_trajectories in num_trajectories_list:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    best_param = None
    max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    for hparam_ucb_scale in hparam_ucb_scale_list:
        # start_time = time.time()

        # print(f"hparam_ucb_scale = {hparam_ucb_scale}")
        args["hparam_ucb_scale"] = hparam_ucb_scale
        
        pool = mp.Pool()
        pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

        reward_mean = np.mean(ep_reward_list)
        reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
        if hparam_ucb_scale <= 8:
            res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_std:0.2f}) "
        else:
            res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_std:0.2f}) "
        # print(f"reward = {reward_mean:0.2f} +/- {reward_std:0.2f}")

        Q_mcts_dict[f"{hparam_ucb_scale}"] = copy.deepcopy(Q_mcts_list)

        if reward_mean > max_reward_mean:
            max_reward_mean = reward_mean 
            best_param = hparam_ucb_scale
    
        ep_reward_list[:] = []
        Q_mcts_list[:] = []
    
        end_time = time.time()
        # print(f"it takes {end_time-start_time:0.4f}")
    
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"
    
    # print(f"max_reward_mean = {max_reward_mean:0.2f}")
    # print(f"it takes {end_time-start_time:0.4f}")
    
    max_reward_mean_list.append(max_reward_mean)
    best_param_list.append(best_param)

In [4]:
print(best_param_list)

[64, 64, 64, 32, 64, 64, 64]


In [5]:
print(res_text1)
print(res_text2)

200 & -101.15 (±0.49) & -96.30 (±21.73) & -76.85 (±43.04) & -52.80 (±49.46) \\ 
 \hline 
500 & -101.05 (±0.22) & -101.10 (±0.31) & -57.40 (±49.56) & -52.50 (±49.76) \\ 
 \hline 
1000 & -96.20 (±21.70) & -96.30 (±21.49) & -71.90 (±45.61) & -38.25 (±47.35) \\ 
 \hline 
1500 & -101.10 (±0.31) & -96.45 (±21.53) & -77.30 (±43.21) & -38.15 (±47.32) \\ 
 \hline 
2000 & -101.10 (±0.45) & -91.35 (±29.70) & -86.50 (±35.56) & -28.50 (±42.95) \\ 
 \hline 
2500 & -101.00 (±0.00) & -101.05 (±0.22) & -81.85 (±39.43) & -33.35 (±45.78) \\ 
 \hline 
3000 & -101.10 (±0.31) & -101.30 (±0.73) & -81.65 (±39.83) & -14.35 (±29.66) \\ 
 \hline 

200 & -42.90 (±48.67) & -28.45 (±42.98) & -18.65 (±35.50) & -23.50 (±39.76) \\ 
 \hline 
500 & -28.55 (±42.92) & -28.25 (±43.09) & -18.55 (±35.54) & -18.55 (±35.54) \\ 
 \hline 
1000 & -23.65 (±39.68) & -8.95 (±21.67) & -8.85 (±21.69) & -8.85 (±21.69) \\ 
 \hline 
1500 & -18.65 (±35.50) & -4.00 (±0.00) & -4.00 (±0.00) & -4.00 (±0.00) \\ 
 \hline 
2000 & -9.20 (±21.62) 